# Rebalance instances

### 1) Overall configurations 

In [1]:
import os
import os.path
import pandas as pd
from pprint import pprint

# Root
root_path = os.getcwd().replace("\\","/")

# Instance settings
# instance_settings_path = "C:/Users/LocalAdmin/IdeaProjects/slevels/src/main/resources/simulation.rebalancing/instance_settings_test_rebalancing.json"
instance_settings_path = "C:/Users/LocalAdmin/IdeaProjects/slevels/src/main/resources/week/allow_hiring.json"

##### Data translation

dict_sq_class = {"A":"Business", "B":"Standard", "C":"Low-cost"}
category_sq_class = pd.api.types.CategoricalDtype(categories=["Business", "Standard", "Low-cost"], ordered=True)

# Vehicle type
dict_service = {"FLEET":"Company-owned", "FREELANCE":"Third-party"}
category_fleet = pd.api.types.CategoricalDtype(categories=["Company-owned", "Third-party"], ordered=True)

# Pickup deadline status (Was user chosen to have its pk deadline extended?)
dict_sl_status = {"MET":"Shortest", "UNMET":"Extended"}
category_status = pd.api.types.CategoricalDtype(categories=["Shortest", "Extended"], ordered=True)

# Establishing category order and alias dictionaries
dict_segmentation = {"AA":"B+", "BB":"S+", "CC":"L+", "A":"B", "B":"S", "C":"L"}
category_segmentation = pd.api.types.CategoricalDtype(categories=["B+", "S+", "L+", "B", "S", "L"], ordered=True)

dict_contract_duration = {3600:"1h", 0:"Single-ride", 10800:"3h", -1:"-"}
category_contract_duration = pd.api.types.CategoricalDtype(categories=["Single-ride", "1h", "3h"], ordered=True)

dict_service_rate = {"S1":"SR1", "S2":"SR2", "S3":"SR3", "BASELINE":"Baseline"}
category_service_rate = pd.api.types.CategoricalDtype(categories=["SR1", "SR2", "SR3"], ordered=True)


### 2) Loading the instance data

In [2]:
import json

def load_json(path):
    """Read json file and return dictionary"""

    # Add .json to the end of file if needed
    if path.find(".json") < 0:
        path = path + ".json"

    # Read JSON file
    with open(path) as data_file:
        data_loaded = json.load(data_file)

    return data_loaded

instances_dic = load_json(instance_settings_path)

# Folder where results will be saved
result_folder = instances_dic["result_folder"]

print("########### INSTANCE SETTINGS ##################################################")
pprint(instances_dic)

########### INSTANCE SETTINGS ##################################################
{'instance_description': 'Whole week experiment (hiring and deteriorating '
                         'enabled)',
 'instance_name': 'HIRINGWEEKMAXWAITINGREB2',
 'instances_folder': 'C:/Users/LocalAdmin/IdeaProjects/slevels/instance_output/week/',
 'labels': {'BA': 'batch_duration',
            'CD': 'contract_duration',
            'CS': 'customer_segmentation',
            'CT': 'clear_target_list_every_round',
            'ID': 'instance_description',
            'IF': 'initial_fleet',
            'IN': 'instance_name',
            'MC': 'max_capacity',
            'MO': 'allow_many_to_one',
            'MR': 'max_requests',
            'RE': 'rebalance',
            'RT': 'reinsert_targets',
            'SD': 'allow_service_deterioration',
            'SR': 'service_rate',
            'ST': 'simulation_time',
            'UR': 'allow_urgent_relocation',
            'VH': 'allow_vehicle_hiring'},
 'rebala

### Get settings from instance name

E.g.:

* Input = `IN-instanceName_BA-30_TH-86400_MR-1000_IF-1000_MC-06_CD-3600-SR-S1_CS-AA_SD_VH_MO_RT_CT_UR`

* Output = 
{allow_many_to_one: True,
allow_service_deterioration: True,
allow_urgent_relocation: True,
allow_vehicle_hiring: True,
batch_duration: 30,
clear_target_list_every_round: True,
contract_duration: 3600,
customer_segmentation: AA,
initial_fleet: 1000,
max_capacity: 06,
max_requests: 1000,
reinsert_targets: True,
time_horizon: 86400}

In [3]:
def get_instance_settings(file_name):
    """ Read file name and return instance settings.
    E.g.:
     Input = IN-instanceName_BA-30_TH-86400_MR-1000_IF-1000_MC-06_CD-3600-SR-S1_CS-AA_SD_VH_MO_RT_CT_UR
     Output = {'allow_many_to_one': True,
                'allow_service_deterioration': True,
                'allow_urgent_relocation': True,
                'allow_vehicle_hiring': True,
                'batch_duration': '30',
                'clear_target_list_every_round': True,
                'contract_duration': '3600',
                'customer_segmentation': 'AA',
                'initial_fleet': '1000',
                'max_capacity': '06',
                'max_requests': '1000',
                'reinsert_targets': True,
                'time_horizon': '86400'}
    """
    label_setting_dic = instances_dic["labels"]
    
    # print(file_name)
        
    # E.g., ['BA-30', 'TH-86400', 'MR-1000', 'IF-1000', 'MC-06', 'CD-3600-SR-S1', 'CS-AA', 'SD', 'VH', 'MO', 'RT', 'CT', 'UR']
    file_instances = file_name.split("_")

    instance_settings = dict()

    for e in file_instances:
        
        if e in label_setting_dic.keys():
            # E.g., e =  SD
            k = label_setting_dic[e]
            # E.g., k = allow_service_deterioration
            instance_settings[k] = True
            
        else:
            # E.g., lv = ["BA", "30"]
            lv  = e.split('-')
            # E.g., e2 = BA
            e2 = lv[0]
            # E.g., k = batch_duration
            k2 = label_setting_dic[e2]
            
            if len(lv) > 1:
                 # E.g., v = '30'
                v = lv[1]
                instance_settings[k2] = v
            else:
                # label is not in instance name = False
                instance_settings[k] = False

    return instance_settings

### Aggregate results (folder request_track)

* `earliest`
* `id` = 1, 2, 3, ..., #USERS
* `class` = A, B, C
* `pk_delay`
* `ride_delay`
* `pk_time`
* `dp_time`
* `id_from` = Network id
* `id_to` = Network id
* `dist` = trip(id_from, id_to) in seconds
* `service` = {FLEET, FREELANCE}
* `service_level` = {MET, UNMET}

In [4]:
import pandas as pd
from pprint import pprint

def get_request_track_dic(path_experiment, name_experiment):
    
    # Load results
    experiment_file = "{}request_track/{}.csv".format(path_experiment, name_experiment)
    
    # print("Processing experiment file '{}'".format(experiment_file))
    df = pd.read_csv(experiment_file, index_col="earliest",  parse_dates = True)
    
    aggfunc = {"pk_delay" : ['mean', 'count', 'max']}
    
    dfp = df.pivot_table(index="class", columns="service_level", aggfunc=aggfunc, values=["pk_delay"])

    return dfp

### Processing all instances in folder (mean time)

In [5]:
from collections import defaultdict
import math


request_log_folder = result_folder + "request_track/"

# Get all instances in folder
instance_file_names = os.listdir(request_log_folder)
print("Reading files in folder:", request_log_folder)


instance_attribute = defaultdict(list)
count_lim = 0
for file_name in instance_file_names:
    #if count_lim > 2:
    #    break
    count_lim = count_lim + 1
    instance, extension = file_name.split(".")
    
    
    # Instance settings
    instance_settings = get_instance_settings(instance)
        
    # Filter fixed fleet instances
    #if instance_settings.get('service_rate', '-') == '-':
    #    continue
 
    if instance_settings['instance_name'] == "HIRINGWEEKMAXWAITINGREB2" or 'rebalance' not in instance_settings.keys():
        continue
    
    if instance_settings['customer_segmentation'] in ['A', 'B', 'C']:
        continue
        
    print("  - Processing", instance)
    
    # Load results
    experiment_file = "{}{}.csv".format(request_log_folder, instance)
    df = pd.read_csv(experiment_file, index_col="earliest",  parse_dates = True)

    column = ('', '', 'service_rate')
    # When no service rate is available, instance refers to baseline
    instance_attribute[column].append(instance_settings.get('service_rate', 'BASELINE'))

    column = ('','', 'contract_duration')
    # When no contract duration is available, instance refers to baseline
    instance_attribute[column].append(int(instance_settings.get('contract_duration', '-1')))

    column = ('','', 'customer_segmentation')
    instance_attribute[column].append(instance_settings['customer_segmentation'])

   
    # Convert denied to freelance to show denied under same header of the initial fleet
    df.loc[(df['service']=='DENIED'), 'service'] = 'FLEET'
    
    
    for sq_class in ['A', 'B', 'C']:
        for fleet in ['FLEET', 'FREELANCE']:
            for sl_status in ["MET", "UNMET"]:
                # Categories leading to max, mean and count
                super_indexes = (dict_sq_class[sq_class], dict_service[fleet], dict_sl_status[sl_status])
                
                # Filter dataframe
                filter_sq_sl = (df['service'] == fleet) & (df['class'] == sq_class) & (df['service_level'] == sl_status)
                df_filtered = df.loc[filter_sq_sl]["pk_delay"]
                mean_pk = df_filtered.mean()
                #max_pk = df_filtered.max()
                #count = df_filtered.count()
                
                # Format data
                # mean_pk_min = ("{:.1f}".format(mean_pk/60) if not math.isnan(mean_pk) else "-")
                mean_pk_min = (mean_pk if not math.isnan(mean_pk) else 0)
                #max_pk_min = ("{:.1f}".format(max_pk/60) if not math.isnan(max_pk) else "-")
                
                #count = ("{:.2%}".format(count/total) if not (math.isnan(count) or count==0) else "-")
                
                #instance_attribute[(*super_indexes, 'Max.')].append(max_pk_min)
                instance_attribute[super_indexes].append(mean_pk_min)
                #instance_attribute[(*super_indexes, 'Count')].append(count)

Reading files in folder: C:/Users/LocalAdmin/IdeaProjects/slevels/instance_output/week/request_track/
  - Processing IN-HIRINGWEEKMAXWAITING_BA-30_ST-604800_MR-1000_IF-1000_MC-4_CS-AA_CD-0_SR-S1_VH_SD_RE_CT_RT_UR
  - Processing IN-HIRINGWEEKMAXWAITING_BA-30_ST-604800_MR-1000_IF-1000_MC-4_CS-AA_CD-0_SR-S2_VH_SD_RE_CT_RT_UR
  - Processing IN-HIRINGWEEKMAXWAITING_BA-30_ST-604800_MR-1000_IF-1000_MC-4_CS-AA_CD-10800_SR-S1_VH_SD_RE_CT_RT_UR
  - Processing IN-HIRINGWEEKMAXWAITING_BA-30_ST-604800_MR-1000_IF-1000_MC-4_CS-AA_CD-10800_SR-S2_VH_SD_RE_CT_RT_UR
  - Processing IN-HIRINGWEEKMAXWAITING_BA-30_ST-604800_MR-1000_IF-1000_MC-4_CS-AA_CD-3600_SR-S1_VH_SD_RE_CT_RT_UR
  - Processing IN-HIRINGWEEKMAXWAITING_BA-30_ST-604800_MR-1000_IF-1000_MC-4_CS-AA_CD-3600_SR-S2_VH_SD_RE_CT_RT_UR
  - Processing IN-HIRINGWEEKMAXWAITING_BA-30_ST-604800_MR-1000_IF-1000_MC-4_CS-BB_CD-0_SR-S1_VH_SD_RE_CT_RT_UR
  - Processing IN-HIRINGWEEKMAXWAITING_BA-30_ST-604800_MR-1000_IF-1000_MC-4_CS-BB_CD-0_SR-S2_VH_SD_RE_CT_RT

### Table: What is the service level (pickup delay) of the users lying outside SQ-class service rate?

In [6]:
from datetime import datetime, timedelta
#pprint(instance_attribute)

# Build data frame from dictionary
a = pd.DataFrame.from_dict(instance_attribute)

key_sr = ('','',  'service_rate') # (S1, S2, S3)
key_cs = ('','',  'customer_segmentation') # (A, AA, BB, etc.)
key_cd = ('','',  'contract_duration') #(0, 3600, 18000)

# Filtering data (only mixed segmentations scenarios will be used)
a = a[a[key_cs].isin(["AA", "BB", "CC"])]

# Renaming data and applying aliases
a[key_cs] = a[key_cs].map(lambda e:dict_segmentation[e])
a[key_cs] = a[key_cs].astype(category_segmentation)

a[key_cd] = a[key_cd].map(lambda e:dict_contract_duration[e])
a[key_cd] = a[key_cd].astype(category_contract_duration)

a[key_sr] = a[key_sr].map(lambda e:dict_service_rate[e])
a[key_sr] = a[key_sr].astype(category_service_rate)

a = a.sort_values(by=[key_cs,key_cd, key_sr])
a = a.set_index([key_cs,key_cd, key_sr])

# Changing table column index names
a.index.names = ["User base", "Contract duration", "Service rate"]

# Transforming to minutes
#a = a.applymap(lambda e: ('{:.2f}'.format(float(e)/60) if e != '-' else e))
a = a.applymap(lambda e: ('{:02}:{:02}'.format(int(e)//60, int(e)%60) if e != 0 else '-'))
a

Business                       \
                                         Company-owned          Third-party   
                                              Shortest Extended    Shortest   
User base Contract duration Service rate                                      
B+        Single-ride       SR1                  02:02        -       02:01   
                            SR2                  02:02    04:49       02:01   
          1h                SR1                  01:53        -       02:06   
                            SR2                  01:54    04:01       02:07   
          3h                SR1                  01:39        -       01:46   
                            SR2                  01:40    03:55       01:47   
          NaN               NaN                  04:01        -           -   
                            NaN                  02:03        -           -   
S+        Single-ride       SR1                  01:50        -       02:09   
                            SR2                  01:50    04:57       02:09   
          1h                SR1                  01:48        -       02:07   
                            SR2                  01:49    04:46       02:07   
          3h                SR1                  01:44        -       01:58   
                            SR2                  01:44    04:36       01:58   
          NaN               NaN                  02:47        -           -   
                            NaN                  01:50        -           -   
L+        Single-ride       SR1                  01:50        -       02:09   
                            SR2                  01:50    04:56       02:09   
          1h                SR1                  01:50        -       02:08   
                            SR2                  01:50    04:49       02:08   
          3h                SR1                  01:46        -       01:59   
                            SR2                  01:46    04:46       02:00   
          NaN               NaN                  02:45        -           -   
                            NaN                  01:51        -           -   

                                                       Standard           \
                                                  Company-owned            
                                         Extended      Shortest Extended   
User base Contract duration Service rate                                   
B+        Single-ride       SR1                 -         02:38    07:20   
                            SR2             01:56         02:38    07:36   
          1h                SR1                 -         02:05    06:57   
                            SR2             04:02         02:07    06:34   
          3h                SR1                 -         01:44    07:00   
                            SR2             03:52         01:45    06:21   
          NaN               NaN                 -         04:57        -   
                            NaN                 -         02:38        -   
S+        Single-ride       SR1                 -         01:57    08:18   
                            SR2             02:09         01:58    08:19   
          1h                SR1                 -         01:45    07:04   
                            SR2             05:02         01:46    07:11   
          3h                SR1                 -         01:38    06:50   
                            SR2             04:46         01:39    06:41   
          NaN               NaN                 -         02:44        -   
                            NaN                 -         01:57        -   
L+        Single-ride       SR1                 -         02:05    08:27   
                            SR2             02:09         02:05    08:16   
          1h                SR1                 -         01:56    06:48   
                            SR2             04:44         01:57    07:29   
         

### Getting latex table (median delay)

In [7]:
print(a.to_latex(multicolumn=True, multirow=True))

\begin{tabular}{lllllllllllllll}
\toprule
   &     &     & \multicolumn{4}{l}{Business} & \multicolumn{4}{l}{Standard} & \multicolumn{4}{l}{Low-cost} \\
   &     &     & \multicolumn{2}{l}{Company-owned} & \multicolumn{2}{l}{Third-party} & \multicolumn{2}{l}{Company-owned} & \multicolumn{2}{l}{Third-party} & \multicolumn{2}{l}{Company-owned} & \multicolumn{2}{l}{Third-party} \\
   &     &     &      Shortest & Extended &    Shortest & Extended &      Shortest & Extended &    Shortest & Extended &      Shortest & Extended &    Shortest & Extended \\
User base & Contract duration & Service rate &               &          &             &          &               &          &             &          &               &          &             &          \\
\midrule
\multirow{8}{*}{B+} & \multirow{2}{*}{Single-ride} & SR1 &         02:02 &        - &       02:01 &        - &         02:38 &    07:20 &       02:58 &        - &         03:24 &        - &       01:12 &        - \\
   &     & SR2 &

### Processing all instances (count)

In [8]:
from collections import defaultdict
import math


request_log_folder = result_folder + "request_track/"

# Get all instances in folder
instance_file_names = os.listdir(request_log_folder)
print("Reading files in folder:", request_log_folder)


instance_attribute_count = defaultdict(list)
instance_attribute_count_percentage = defaultdict(list)
count_lim = 0
for file_name in instance_file_names:
    #if count_lim > 0:
    #    break
    count_lim = count_lim + 1
    instance, extension = file_name.split(".")
    
    
    # Instance settings
    instance_settings = get_instance_settings(instance)
        
    # Filter fixed fleet instances
    #if instance_settings.get('service_rate', '-') == '-':
    #    continue
        
    if instance_settings['instance_name'] == "HIRINGWEEKMAXWAITINGREB2" or 'rebalance' not in instance_settings.keys():
        continue
    
    if instance_settings['customer_segmentation'] in ['A', 'B', 'C']:
        continue
        
    #if instance_settings['instance_name'] != "WEEKDENY" or 'rebalance' not in instance_settings.keys():
    #    continue
    
    print("  - Processing", instance)
    
    try:
        # Load results
        experiment_file = "{}{}.csv".format(request_log_folder, instance)
        df = pd.read_csv(experiment_file, index_col="earliest",  parse_dates = True)

        column = ('', '', 'service_rate')
        # When no service rate is available, instance refers to baseline
        instance_attribute_count[column].append(instance_settings.get('service_rate', 'BASELINE'))
        instance_attribute_count_percentage[column].append(instance_settings.get('service_rate', 'BASELINE'))
        
        column = ('','', 'contract_duration')
        # When no contract duration is available, instance refers to baseline
        instance_attribute_count[column].append(int(instance_settings.get('contract_duration', '-1')))
        instance_attribute_count_percentage[column].append(int(instance_settings.get('contract_duration', '-1')))
        
        column = ('','', 'customer_segmentation')
        instance_attribute_count[column].append(instance_settings['customer_segmentation'])
        instance_attribute_count_percentage[column].append(instance_settings['customer_segmentation'])
        
        # Get total number of requests
        total = len(df)
        
        # Convert denied to freelance to show denied under same header of the initial fleet
        df.loc[(df['service']=='DENIED'), 'service'] = 'FLEET'
        
        print(df.service.value_counts())

        for sq_class in ['A', 'B', 'C']:
            for fleet in ['FLEET', 'FREELANCE']:
                for sl_status in ["MET", "UNMET"]:

                    # Categories leading to max, mean and count
                    #super_indexes = (dict_service[fleet], dict_sq_class[sq_class], dict_sl_status[sl_status])
                    #super_indexes = (dict_service[fleet], dict_sl_status[sl_status], dict_sq_class[sq_class])
                    super_indexes = (dict_sq_class[sq_class], dict_service[fleet], dict_sl_status[sl_status])

                    # Filter dataframe
                    filter_sq_sl = (df['service'] == fleet) & (df['class'] == sq_class) & (df['service_level'] == sl_status)
                    df_filtered = df.loc[filter_sq_sl]["pk_delay"]
                    
                        
                    mean_pk = df_filtered.mean()
                    max_pk = df_filtered.max()
                    
                    count = df_filtered.count()
                    instance_attribute_count[super_indexes].append(count)

                    # Format data
                    # mean_pk_min = ("{:.1f}".format(mean_pk/60) if not math.isnan(mean_pk) else "-")
                    # mean_pk_min = ("{:.1f}".format(int(mean_pk)) if not math.isnan(mean_pk) else "-")
                    #max_pk_min = ("{:.1f}".format(max_pk/60) if not math.isnan(max_pk) else "-")

                    count = (count/total if not (math.isnan(count) or count==0) else 0)
                    instance_attribute_count_percentage[super_indexes].append(count)
                    
                    #instance_attribute[(*super_indexes, 'Max.')].append(max_pk_min)
                    # instance_attribute[(*super_indexes, 'Mean.')].append(mean_pk_min)
                    
                    

                # Adding subtotal
                #filter_fleet_sl_status =(df['service'] == fleet) & (df['service_level'] == sl_status)
                #sub_fleet_sl_status = (dict_service[fleet], dict_sl_status[sl_status], 'Total')
                #count_fleet_sl = df.loc[filter_fleet_sl_status]["pk_delay"].count()
                #count_fleet_sl = (count_fleet_sl/total if not (math.isnan(count_fleet_sl) or count_fleet_sl==0) else 0)
                #instance_attribute_count[sub_fleet_sl_status].append(count_fleet_sl)

                # Adding total
                subtotal_index = (dict_sq_class[sq_class], dict_service[fleet], 'Total')
               
                count_fleet = len(df.loc[(df['service'] == fleet) & (df['class'] == sq_class)])
                instance_attribute_count[subtotal_index].append(count_fleet)
                
                count_fleet = (count_fleet/total if not (math.isnan(count_fleet) or count_fleet==0) else 0)
                instance_attribute_count_percentage[subtotal_index].append(count_fleet)
                
    except Exception as e:
        print(e)
        pass

Reading files in folder: C:/Users/LocalAdmin/IdeaProjects/slevels/instance_output/week/request_track/
  - Processing IN-HIRINGWEEKMAXWAITING_BA-30_ST-604800_MR-1000_IF-1000_MC-4_CS-AA_CD-0_SR-S1_VH_SD_RE_CT_RT_UR
FLEET        1415111
FREELANCE     169961
Name: service, dtype: int64
  - Processing IN-HIRINGWEEKMAXWAITING_BA-30_ST-604800_MR-1000_IF-1000_MC-4_CS-AA_CD-0_SR-S2_VH_SD_RE_CT_RT_UR
FLEET        1411447
FREELANCE     173625
Name: service, dtype: int64
  - Processing IN-HIRINGWEEKMAXWAITING_BA-30_ST-604800_MR-1000_IF-1000_MC-4_CS-AA_CD-10800_SR-S1_VH_SD_RE_CT_RT_UR
FLEET        1159356
FREELANCE     425716
Name: service, dtype: int64
  - Processing IN-HIRINGWEEKMAXWAITING_BA-30_ST-604800_MR-1000_IF-1000_MC-4_CS-AA_CD-10800_SR-S2_VH_SD_RE_CT_RT_UR
FLEET        1160662
FREELANCE     424410
Name: service, dtype: int64
  - Processing IN-HIRINGWEEKMAXWAITING_BA-30_ST-604800_MR-1000_IF-1000_MC-4_CS-AA_CD-3600_SR-S1_VH_SD_RE_CT_RT_UR
FLEET        1238478
FREELANCE     346594
Name: serv

### Table: How many users were serviced by each vehicle type?

In [9]:
# Build data frame from dictionary
b = pd.DataFrame.from_dict(instance_attribute_count_percentage)

key_sr = ('','','service_rate') # (S1, S2, S3)
key_cs = ('','',  'customer_segmentation') # (A, AA, BB, etc.)
key_cd = ('','',  'contract_duration') #(0, 3600, 18000)

# Filtering data (only mixed segmentations scenarios will be used)
b = b[b[key_cs].isin(["AA", "BB", "CC"])]

# Renaming data and applying aliases
b[key_cs] = b[key_cs].map(lambda e:dict_segmentation[e])
b[key_cs] = b[key_cs].astype(category_segmentation)

b[key_cd] = b[key_cd].map(lambda e:dict_contract_duration[e])
b[key_cd] = b[key_cd].astype(category_contract_duration)

b[key_sr] = b[key_sr].map(lambda e:dict_service_rate[e])
b[key_sr] = b[key_sr].astype(category_service_rate)

#b = b.sort_values(by=[key_sr, key_cs, key_cd])
#b = b.set_index([key_sr, key_cs, key_cd])

# Changing table column index names
# b.index.names = ["Service rate","User base","Contract duration"]

#b = b.sort_values(by=[key_cd,  key_cs, key_sr])
#b = b.set_index([key_cd,  key_cs, key_sr])

# Changing table column index names
#b.index.names = ["Contract duration","User base", "Service rate"]

b = b.sort_values(by=[key_cs,key_cd, key_sr])
b = b.set_index([key_cs,key_cd, key_sr])

# Changing table column index names
b.index.names = ["User base", "Contract duration", "Service rate"]

def formatResult(e):
    if e == 0:
        return '-'
    if e <= 0.0001:
        return '*'
    return '{:.2%}'.format(float(e))
    
extended = pd.DataFrame(b)
# Transforming to minutes
b = b.applymap(lambda e: ('{:.4%}'.format(float(e)) if e != 0 else '-'))
b= b[[(a, b, c) for a in ['Business', 'Standard', 'Low-cost'] for b in ['Company-owned', 'Third-party'] for c in ['Shortest','Extended']]]

In [10]:
print(b.to_latex(multicolumn=True, multirow=True))

\begin{tabular}{lllllllllllllll}
\toprule
   &     &     & \multicolumn{4}{l}{Business} & \multicolumn{4}{l}{Standard} & \multicolumn{4}{l}{Low-cost} \\
   &     &     & \multicolumn{2}{l}{Company-owned} & \multicolumn{2}{l}{Third-party} & \multicolumn{2}{l}{Company-owned} & \multicolumn{2}{l}{Third-party} & \multicolumn{2}{l}{Company-owned} & \multicolumn{2}{l}{Third-party} \\
   &     &     &      Shortest &  Extended &    Shortest & Extended &      Shortest & Extended &    Shortest & Extended &      Shortest & Extended &    Shortest & Extended \\
User base & Contract duration & Service rate &               &           &             &          &               &          &             &          &               &          &             &          \\
\midrule
\multirow{8}{*}{B+} & \multirow{2}{*}{Single-ride} & SR1 &      57.2854\% &         - &    10.7103\% &        - &      15.9941\% &  0.0011\% &     0.0105\% &        - &      15.9966\% &        - &     0.0018\% &        - \\
   &  

In [11]:
b.to_csv("Counts.csv")

In [12]:
file = "IN-WEEKDENY_BA-30_ST-604800_MR-1000_IF-1000_MC-4_CS-AA_RE_CT_RT_UR.csv"

user = pd.read_csv(experiment_file, index_col="earliest",  parse_dates = True, comment="#")

In [13]:
user.service_level.value_counts()

MET      1548195
UNMET      36877
Name: service_level, dtype: int64

In [14]:
if 'Company-ownedd' in ('Low-cost', 'Company-owned', 'Extended'):
    print("e")

In [15]:
c = pd.DataFrame(b)

In [16]:
extended[[(a, b, c) for a in ['Business', 'Standard', 'Low-cost'] for b in ['Company-owned', 'Third-party'] for c in ['Extended']]]

Business              \
                                         Company-owned Third-party   
                                              Extended    Extended   
User base Contract duration Service rate                             
B+        Single-ride       SR1               0.000000    0.000000   
                            SR2               0.012140    0.000029   
          1h                SR1               0.000000    0.000000   
                            SR2               0.002219    0.001013   
          3h                SR1               0.000000    0.000000   
                            SR2               0.000982    0.000459   
          NaN               NaN               0.085701    0.000000   
                            NaN               0.106694    0.000000   
S+        Single-ride       SR1               0.000000    0.000000   
                            SR2               0.002204    0.000315   
          1h                SR1               0.000000    0.000000   
                            SR2               0.000771    0.000222   
          3h                SR1               0.000000    0.000000   
                            SR2               0.000428    0.000160   
          NaN               NaN               0.012169    0.000000   
                            NaN               0.023107    0.000000   
L+        Single-ride       SR1               0.000000    0.000000   
                            SR2               0.001814    0.000527   
          1h                SR1               0.000000    0.000000   
                            SR2               0.000722    0.000143   
          3h                SR1               0.000000    0.000000   
                            SR2               0.000394    0.000096   
          NaN               NaN               0.013332    0.000000   
                            NaN               0.021186    0.000000   

                                              Standard                \
                                         Company-owned   Third-party   
                                              Extended      Extended   
User base Contract duration Service rate                               
B+        Single-ride       SR1           1.135593e-05  0.000000e+00   
                            SR2           2.712805e-05  0.000000e+00   
          1h                SR1           3.154425e-06  0.000000e+00   
                            SR2           6.308850e-07  6.308850e-07   
          3h                SR1           2.523540e-06  0.000000e+00   
                            SR2           2.523540e-06  6.308850e-07   
          NaN               NaN           8.832390e-06  0.000000e+00   
                            NaN           8.012239e-05  0.000000e+00   
S+        Single-ride       SR1           5.387758e-04  6.308850e-07   
                            SR2           1.302147e-03  6.308850e-07   
          1h                SR1           6.435027e-05  9.463275e-06   
                            SR2           1.678154e-04  2.523540e-05   
          3h                SR1           2.460451e-05  3.154425e-06   
                            SR2           4.731637e-05  7.570620e-06   
          NaN               NaN           1.486365e-03  0.000000e+00   
                            NaN           5.129726e-03  0.000000e+00   
L+        Single-ride       SR1           2.252259e-04  5.047080e-06   
                            SR2           4.649622e-04  9.463275e-06   
          1h                SR1           2.397363e-05  1.892655e-06   
                            SR2           5.173257e-05  3.154425e-06   
          3h                SR1           6.308850e-06  1.261770e-06   
                            SR2           1.514124e-05  6.308850e-07   
          NaN               NaN           1.760169e-03  0.000000e+00   
                            NaN           1.969623e-03  0.000000e+00   

                                              Low-cost              